<a href="https://colab.research.google.com/github/ethantlogan/Homework_3/blob/main/Homework4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#This code imports the data from my drive into a pandas dataframe and uploads it
from google.colab import drive
drive.mount('/content/drive')

filedir = '/content/drive/MyDrive/FWE458/'
filename = filedir + "Wcr_GPPdaily.csv"

import pandas as pd
DF = pd.read_csv(filename)
DF

Mounted at /content/drive


,TIMESTAMP,TA_F,SW_IN_F,VPD_F,GPP_NT_VUT_REF
0,19990101,-20.063,72.603,0.413,-0.517364
1,19990102,-12.814,12.358,0.147,-0.094241
2,19990103,-12.625,33.132,0.128,-0.166819
3,19990104,-18.652,93.481,0.263,-0.582301
4,19990105,-20.269,45.502,0.261,-0.568240
...,...,...,...,...,...
5839,20141227,-5.112,22.956,0.497,0.195777
5840,20141228,-12.408,33.332,0.531,0.093025
5841,20141229,-18.204,69.597,0.461,-0.234488
5842,20141230,-22.252,81.959,0.448,-0.315846


In [ ]:
#This command renames all of the columns by assigning them new names. The inplace
#makes it so this edits the original dataframe and I do not have to create a new
#dataframe
DF.rename(columns = {"TA_F" : "TA", "SW_IN_F" : "SW", "VPD_F" : "VPD", "GPP_NT_VUT_REF" : "GPP"}, inplace=True)
DF

,TIMESTAMP,TA,SW,VPD,GPP
0,19990101,-20.063,72.603,0.413,-0.517364
1,19990102,-12.814,12.358,0.147,-0.094241
2,19990103,-12.625,33.132,0.128,-0.166819
3,19990104,-18.652,93.481,0.263,-0.582301
4,19990105,-20.269,45.502,0.261,-0.568240
...,...,...,...,...,...
5839,20141227,-5.112,22.956,0.497,0.195777
5840,20141228,-12.408,33.332,0.531,0.093025
5841,20141229,-18.204,69.597,0.461,-0.234488
5842,20141230,-22.252,81.959,0.448,-0.315846


In [ ]:
#The drop command essentially edits the dataframe to get rid of a certain
#columnn/row, decided by the axis command, so axis=1 means it's a
#column. Again the inplace means it edits the original dataframe
DF.drop("TIMESTAMP", axis = 1, inplace = True)
DF

,TA,SW,VPD,GPP
0,-20.063,72.603,0.413,-0.517364
1,-12.814,12.358,0.147,-0.094241
2,-12.625,33.132,0.128,-0.166819
3,-18.652,93.481,0.263,-0.582301
4,-20.269,45.502,0.261,-0.568240
...,...,...,...,...
5839,-5.112,22.956,0.497,0.195777
5840,-12.408,33.332,0.531,0.093025
5841,-18.204,69.597,0.461,-0.234488
5842,-22.252,81.959,0.448,-0.315846


In [ ]:
#This print commands scans all of the data to see if there is any "null" AKA
#missing data. Since the print returns "false" that means there is not missing
#data and no interpolation needs to happen.
print(DF.isnull().values.any())

False


In [ ]:
#These commands create new columns to the dataframe by multiplying already
#existing columns.
DF["SWxVPD"] = DF["SW"] * DF["VPD"]
DF["SWxTA"] = DF["SW"] * DF["TA"]
DF["VPDxTA"] = DF["VPD"] * DF["TA"]
DF

,TA,SW,VPD,GPP,SWxVPD,SWxTA,VPDxTA
0,-20.063,72.603,0.413,-0.517364,29.985039,-1456.633989,-8.286019
1,-12.814,12.358,0.147,-0.094241,1.816626,-158.355412,-1.883658
2,-12.625,33.132,0.128,-0.166819,4.240896,-418.291500,-1.616000
3,-18.652,93.481,0.263,-0.582301,24.585503,-1743.607612,-4.905476
4,-20.269,45.502,0.261,-0.568240,11.876022,-922.280038,-5.290209
...,...,...,...,...,...,...,...
5839,-5.112,22.956,0.497,0.195777,11.409132,-117.351072,-2.540664
5840,-12.408,33.332,0.531,0.093025,17.699292,-413.583456,-6.588648
5841,-18.204,69.597,0.461,-0.234488,32.084217,-1266.943788,-8.392044
5842,-22.252,81.959,0.448,-0.315846,36.717632,-1823.751668,-9.968896


In [ ]:
#First I needed to import all of the tools I would be using
import pandas as pd
import numpy as np
from scipy.optimize import minimize
#Then I had to define the linear model with inputs coming from all of the
#variables that needed to be included in the linear model
def linear_model(params, SW, VPD, TA, SWxVPD, SWxTA, VPDxTA):
  #Then I had assign variables to being different parameters, or constants, that
  #define the relationship between the independent and dependent variables
  β0, β1, β2, β3, β4, β5, β6 = params
  #I then created the linear function using the template from Canvas
  GPP = β0 + β1 * SW + β2 * VPD + β3 * TA + β4 * (SWxVPD) + β5 * (SWxTA) + β6 * (VPDxTA)
  #Returning the GPP assigns that dependent variable a value.
  return GPP

#I then define a new function to get the value for the Mean Squared Error from
#the set parameters and the data in the data set.
def MSE(params, SW, VPD, TA, GPP, SWxVPD, SWxTA, VPDxTA):
  #The y prediction predicts values for all of the independent variables from
  #the list in order to be able to calculate Mean Squared Error from the guessed
  #data to the predicted data
  y_pred = linear_model(params, SW, VPD, TA, SWxVPD, SWxTA, VPDxTA)
  #This function then does the math to calculate the MSE
  mse = np.mean((GPP - y_pred)**2)
  return mse

#All of these functions just create shortcuts for getting the values for the
#independent variables from the original dataframe
SW = DF["SW"].values
VPD = DF["VPD"].values
TA = DF["TA"].values
GPP = DF["GPP"].values
SWxVPD = DF["SWxVPD"].values
SWxTA = DF["SWxTA"].values
VPDxTA = DF["VPDxTA"].values

#This function just makes a guess at the values for the variables, starting at
#0 which is a common starting guess.
initial_guess = np.zeros(7)

#This uses the minimize function of scipy to build a linear regression model
#from the MSE, guesses of 0, and the arguments in the form of the independent
#variables
result = minimize(MSE, initial_guess, args = (SW, VPD, TA, GPP, SWxVPD, SWxTA, VPDxTA))

#This defines the optimized coefficients as a function of the minimize function
#and the data of the dependent variable.
optimized_coefficints = result.x

#The f in front of the rest of the text makes it so it runs the function and
#displays the results from that function, and not exactly what the text says
print(f"Optimized Coefficients : {optimized_coefficints}")


Optimized Coefficients : [-9.44639139e+02 -3.24582901e+00 -6.48394726e+02  8.21507901e+01
  1.79162425e+00 -4.56008271e-01  1.35971987e+01]


In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

#This defines the indepedent and dependent variables to their respective axes
x = DF[["SW", "VPD", "TA", "SWxVPD", "SWxTA", "VPDxTA"]]
y = DF["GPP"]

#This then defines the model to a Linear Regression model and fits the data
#from the dataframe to the linear regression model to create data points and
#a fit line
model = LinearRegression()
model.fit(x, y)

#This prints the outcomes of the minimized coefficients in a neat way
print(f"Intercept (β0): {model.intercept_}")
print(f"Coefficients (β1, β2, β3, β4, β5, β6): {model.coef_}")

Intercept (β0): -944.0672018203164
Coefficients (β1, β2, β3, β4, β5, β6): [-3.24914968e+00 -6.48574173e+02  8.21699813e+01  1.79249151e+00
 -4.56050357e-01  1.35954025e+01]


In [ ]:
#These are predictions from the SciPy optimization (minimize) model
predictions_scipy = linear_model(result.x, SW, VPD, TA, SWxVPD, SWxTA, VPDxTA)


#This function is just creating the R2 formula on my own by getting the sum
#of the total squares, and of the residuals from the true values - the predicted
#values
def R_squared(y_true, y_pred):
    ss_total = np.sum((y_true - np.mean(y_true)) ** 2)  # Total sum of squares
    ss_residual = np.sum((y_true - y_pred) ** 2)  # Residual sum of squares
    R2 = 1 - (ss_residual / ss_total)  # R² formula
    return R2

#Then this function actually calculates the R2 by using the GPP value (the
#outcome of the linear model) and the predicted values for the GPP from the
#independent variables and using scipy to do the rest of the math.
r2_scipy = R_squared(GPP, predictions_scipy)
print(f"R2 for SciPy Optimization Model: {r2_scipy}")

R2 for SciPy Optimization Model: 0.01358269810244761


In [ ]:
#This is a built in calculation into sklearn that automatically calculates an R2
#value from the linear model built before.
R2_sklearn = model.score(x, y)
print(f"R2 for sklearn Linear Regression Model: {R2_sklearn}")

R2 for sklearn Linear Regression Model: 0.013582700933254976


R2 values are very close, as well as all of the coefficient values. The R2 value for sklearn is slightly higher, as well as all of the coefficient values are slightly higher except for β6. Both models are so close though that the differences do not matter much.